## README

This script crops the arena pictures of each mouse track experiment.

It is advisable to preprocess (e.g., in Photoshop) the arena pictures that will be processed by this script.
- The original arena picture must keep all the landmarks but have its B&W contrast enhanced
- A copy of the original arena picture with digitally removed landmarks is advisable to make hole identification correct.
- Adjust the correction pixels in the tuple list of picture files below (last 3 values) to get better alignment results.

The hole detection may detect multiple times a single hole, you must correct for that in the generated txt file by checking the hole index in the _holes.png gerenated picture
Otherwise, you can digitally paint the faulty holes in photoshop before processing the arena picture with this script

When files with _entrance.png are provided (entrance marks in black on a completely white background; obtained from the original picture),
then the entrance coordinates are also written in the _holes.txt file using the special hole indices -4 to -1

The idea here is to obtain an arena picture that is compatible with the arena picture of the first experiments (the "pilot" pictures in the main arena_picture folder)

Outputs:
    * a cropped arena picture with an img_extent
        this file should be included in the process_mouse_trial_lib function "get_arena_picture_bounds"
        with the proper "if" condition for that experiment (which is usually copied from Kelly's plot script, e.g. the experimemnt date most of the times)
    * the img_extent for the cropped picture (printed on the screen)
    * a BW arena picture with the (0,0)_world coordinates marked in blue
    * a BW arena picture with all the detected holes identified by unique sequential indices
    * a txt file with the (x,y)_world coordinates of each identified hole that is marked in the holes picture

The (x0,x1)_world and (y0,y1)_world parameters in the list below are also copied from Kelly's plot script, and represent the original extent (or x,y axes bounds) of each full arena picture.



## #################################################
## #################################################
## #################################################
## #################################################

## TO DO
update the algorithm such that it reads some "*_marks.png" arena files, where the marks contain each object in a different color
e.g.: arena boundarys in blue; entrance in red, green, magenta and cyan; targets in subtones of these colors corresponding to each entrance, holes color, etc...
then, I can use the cluster-finding code to seek for the cluster of each specific color, and I don't need to use the circle detection algorightm
it could have some input like this:
'arena_picture/dec-2019/BKGDimage-localCues.png',                      'arena_picture/dec-2019/BKGDimage-localCues_preproc.png',        misc.structtype(arena=(0,0,255),holes=(0,0,0),entrances={0:(255,0,0),1:(0,255,0),2:(255,0,255),3:(0,255,255)},targets={0:(150,0,0),1:(0,150,0),2:(150,0,150),3:(0,150,150)})

In [1]:
import os
import numpy
import modules.io as io
import modules.edge_detector as edged
import modules.helper_func_class as misc
import modules.process_mouse_trials_lib as plib
import PIL
import PIL.ImageDraw
import PIL.ImageEnhance

arena_pic_dir         = 'arena_picture'
#arena_pic_dir          = 'debug_arena_picture'
arena_pic_ref          = os.path.join(arena_pic_dir,'BKGDimage-pilot.png') #os.path.join(arena_pic_dir,plib.get_arena_all_picture_filename()['SW'])
arena_ref_img_mark     = os.path.join(arena_pic_dir,'BKGDimage-pilot_mark.png')
arena_pic_bounds       = misc.structtype(arena_pic_left=-97.0,arena_pic_right=97.0, arena_pic_bottom=-73.0,arena_pic_top=73.0) #plib.get_arena_picture_bounds()
arena_radius_tolerance = 0.02 # tolerance for the apparent arena radius in a picture (if ratio of radius/radius_ref > tolerance, means the the picture is zoomed in/out in relation to the reference picture)

delete_before_processing = True

to_gray = lambda I: I.convert('LA') if 'A' in I.getbands() else I.convert('L')

dx_ref = numpy.asarray((arena_pic_bounds.arena_pic_left   , arena_pic_bounds.arena_pic_right ))
dy_ref = numpy.asarray((arena_pic_bounds.arena_pic_bottom ,  arena_pic_bounds.arena_pic_top  ))

"""
the mark_scheme varible represents the colors of each feature to be detected

a file with suffix *_mark.png should be given for the arena picture to be processed according to the mark_scheme;
otherwise, mark_scheme will NOT be used.
"""
mark_scheme = misc.structtype(arena_boundary='#0000FF', holes='#000000', entrances='#00FF00', targets='#FF0000')

"""
The tuple list below contains the following information in each tuple

[ tuple_1, tuple_2, ... ]

such that tuple_X has:

   do_processing,                            # flag to determine whether this file will be processed
   arena_file_name_inside_arena_picture_dir, # relative to the arena_picture directory, this is the file that will be rescaled and used to plot trajectories (must keep landmarks)
   arena_preprocessed_file_name_if_needed,   # (optional, recommended) relative to the arena_picture directory, optional, this file is only used for hole detection, should have landmarks removed (digitally)
   arena_entrance_file,                      # only entrance marks in white background
   arena_marks_file,                         # (optional, recommended; makes the code faster) marks file with marks for entrance, holes, arena circle, and targets, respecting the mark_scheme color scheme
   (x0,x1)_world,                            # the left and right boundaries (in cm) of the original arena picture given
   (y0,y1)_world,                            # the bottom and top boundaries (in cm) of the original arena picture given
   contrast_factor,                          # contrast enhancing factor (adjust at will to get better automatic arena and whole detection)
   brightness_factor,                        # brightness enhancing factor (adjust at will to get better automatic arena and whole detection)
   dx_px_correction,                         # correction factors for the corresponding coordinates in the arena and hole detection (in pixels)
   dy_px_correction,                         # correction factors for the corresponding coordinates in the arena and hole detection (in pixels)
   r_correction                              # correction factors for the corresponding coordinates in the arena and hole detection (in pixels)
"""
arena_pic = [
   (False, 'dec-2019/BKGDimage-localCues.png'                      , 'dec-2019/BKGDimage-localCues_preproc.png'                      , 'dec-2019/BKGDimage-localCues_entrance.png'                      , ''                                                          , numpy.array((-129.58, 129.58)), numpy.array((-73.03, 73.03)),100.0,1.1, 0, 0,-2),
   (False, 'mar-may-2021/BKGDimage-localCues_clear.png'            , 'mar-may-2021/BKGDimage-localCues_clear_preproc.png'            , 'mar-may-2021/BKGDimage-localCues_clear_entrance.png'            , ''                                                          , numpy.array((-151.06, 150.43)), numpy.array((-84.23, 84.86)),100.0,1.1, 0, 0,-4),
   (False, 'mar-may-2021/BKGDimage-localCues_Letter.png'           , 'mar-may-2021/BKGDimage-localCues_Letter_preproc.png'           , 'mar-may-2021/BKGDimage-localCues_Letter_entrance.png'           , ''                                                          , numpy.array((-151.06, 150.43)), numpy.array((-84.23, 84.86)),100.0,1.1, 0, 0,-4),
   (False, 'mar-may-2021/BKGDimage-localCues_LetterTex.png'        , 'mar-may-2021/BKGDimage-localCues_LetterTex_preproc.png'        , 'mar-may-2021/BKGDimage-localCues_LetterTex_entrance.png'        , ''                                                          , numpy.array((-151.06, 150.43)), numpy.array((-84.23, 84.86)),100.0,1.1, 0, 0,-4),
   (False, 'jun-jul-aug-nov-2021/BKGDimage_3LocalCues.png'         , 'jun-jul-aug-nov-2021/BKGDimage_3LocalCues_preproc.png'         , 'jun-jul-aug-nov-2021/BKGDimage_3LocalCues_entrance.png'         , ''                                                          , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1, 0, 0,-4),
   (False, 'jun-jul-aug-nov-2021/BKGDimage_3LocalCues_Reverse.png' , 'jun-jul-aug-nov-2021/BKGDimage_3LocalCues_Reverse_preproc.png' , 'jun-jul-aug-nov-2021/BKGDimage_3LocalCues_Reverse_entrance.png' , ''                                                          , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1, 0, 0,-4),
   (False, 'jun-jul-aug-nov-2021/BKGDimage_asymm.png'              , 'jun-jul-aug-nov-2021/BKGDimage_asymm_preproc.png'              , 'jun-jul-aug-nov-2021/BKGDimage_asymm_entrance.png'              , ''                                                          , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1, 0, 0,-2),
   (False, 'jun-jul-aug-nov-2021/BKGDimage_asymmRev.png'           , 'jun-jul-aug-nov-2021/BKGDimage_asymmRev_preproc.png'           , 'jun-jul-aug-nov-2021/BKGDimage_asymmRev_entrance.png'           , ''                                                          , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1, 0, 0,-2),
   (False, 'jun-jul-aug-nov-2021/BKGDimage-arenaB.png'             , ''                                                              , 'jun-jul-aug-nov-2021/BKGDimage-arenaB_entrance.png'             , 'jun-jul-aug-nov-2021/BKGDimage-arenaB_mark.png'            , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1,-2,-4,-5),
   (False, 'jun-jul-aug-nov-2021/BDGDimage_arenaB_visualCues.png'  , ''                                                              , 'jun-jul-aug-nov-2021/BDGDimage_arenaB_visualCues_entrance.png'  , 'jun-jul-aug-nov-2021/BDGDimage_arenaB_visualCues_mark.png' , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1,-2,-4,-2),
   (False, 'jun-jul-aug-nov-2021/BKGDimage-Nov15.png'              , ''                                                              , 'jun-jul-aug-nov-2021/BKGDimage-Nov15_entrance.png'              , 'jun-jul-aug-nov-2021/BKGDimage-Nov15_mark.png'             , numpy.array((-151.26, 150.94)), numpy.array((-84.66, 85.29)),100.0,1.1,-2,-4,-2),
   (True , '2022/BKGDimage-220812.png'                             , ''                                                              , '2022/BKGDimage-220812_entrance.png'                             , '2022/BKGDimage-220812_mark.png'                            , numpy.array((-151.65, 151.34)), numpy.array((-84.88, 85.51)),100.0,1.1, 0, 0, 0),
   (True , '2022/BKGDimage-20221011.png'                           , ''                                                              , '2022/BKGDimage-20221011_entrance.png'                           , '2022/BKGDimage-20221011_mark.png'                          , numpy.array((-136.92, 136.63)), numpy.array((-76.78, 77.20)),100.0,1.1, 0, 0, 0)
   ]


mark_pic_not_required = [
   'dec-2019/BKGDimage-localCues.png'                     ,
   'mar-may-2021/BKGDimage-localCues_clear.png'           ,
   'mar-may-2021/BKGDimage-localCues_Letter.png'          ,
   'mar-may-2021/BKGDimage-localCues_LetterTex.png'       ,
   'jun-jul-aug-nov-2021/BKGDimage_3LocalCues.png'        ,
   'jun-jul-aug-nov-2021/BKGDimage_3LocalCues_Reverse.png',
   'jun-jul-aug-nov-2021/BKGDimage_asymm.png'             ,
   'jun-jul-aug-nov-2021/BKGDimage_asymmRev.png'          ]

In [2]:
# calculating the center of the arena for the reference image
# in pixels

arena_ref_xyr    = edged.find_arena_in_image('',arena_ref_img_mark,arena_color=mark_scheme.arena_boundary)
arena_center_ref = numpy.array(arena_ref_xyr[:2])
#_,arena_center_ref = plib.get_center_radius_in_px(arena_pic_bounds.arena_pic_left,arena_pic_bounds.arena_pic_right,arena_pic_bounds.arena_pic_bottom,arena_pic_bounds.arena_pic_top,\
#                                         plib.get_arena_diameter_cm() / 2.0, plib.get_arena_center(), plib.get_arena_picture_file_width_height()[0], plib.get_arena_picture_file_width_height()[1])

for do_processing,p,p_preproc,p_entrance,p_mark,dx,dy,contrast_factor,bright_factor,dx_correct,dy_correct,r_correct in arena_pic:
    if not do_processing:
        continue

    has_picture_with_marks    = len(p_mark)>0
    has_picture_with_entrance = len(p_entrance)>0
    has_picture_preproc       = len(p_preproc)>0

    if (not has_picture_with_marks) and not(arena_pic_name in mark_pic_not_required):
        raise RuntimeError('ERROR! - You must make a mark picture for the arena you are trying to crop: %s' % arena_pic_name)

    #p,dx,dy,contrast_factor,bright_factor = arena_pic[0]
    print(' *** processing %s' % p)

    arena_pic_name          = os.path.join(arena_pic_dir,p)
    arena_pic_mark_name     = os.path.join(arena_pic_dir,p_mark)     if has_picture_with_marks    else ''
    arena_pic_entrance_name = os.path.join(arena_pic_dir,p_entrance) if has_picture_with_entrance else ''
    arena_pic_preproc_name  = os.path.join(arena_pic_dir,p_preproc)  if has_picture_preproc       else ''

    cropped_pic_name    = os.path.join(arena_pic_dir,p.replace('.png','_cropped.png'))
    holes_pic_filename  = os.path.join(arena_pic_dir,p.replace('.png','_holes.png'))
    holes_txt_filename  = os.path.join(arena_pic_dir,p.replace('.png','_holes.txt'))
    origin_pic_name     = os.path.join(arena_pic_dir,p.replace('.png','_origin.png'))
    if delete_before_processing:
        for f in [cropped_pic_name,holes_pic_filename,holes_txt_filename,origin_pic_name]:
            if os.path.isfile(f):
                os.remove(f)
    
    get_transformed_picture = lambda pic_name,radius_ref=1.0,radius=1.0: edged.transform_pic_to_match_distance(arena_pic_ref,pic_name,dx_ref,dy_ref,dx,dy,radius_ref,radius)
    to_gray_enhanced        = lambda im: PIL.ImageEnhance.Contrast(PIL.ImageEnhance.Brightness(to_gray(im)).enhance(bright_factor)).enhance(contrast_factor)
    erase_outside_arena     = lambda im,xyr: edged.erase_outside_circle(im, xyr[:2], xyr[2]+r_correct, bgcolor=1.0)
    get_cropped_aligned_pic = lambda im,xyr: edged.align_arena_center_and_crop(arena_pic_ref,im,arena_center_ref,xyr[:2],bgcolor_rgb=None,dx_correct_px=dx_correct,dy_correct_px=dy_correct,xRange_world_ref=dx_ref,xRange_world=dx,yRange_world_ref=dy_ref,yRange_world=dy)
    apply_all_transforms    = lambda pic_name,xyr,radius_ref=1.0,radius=1.0: get_cropped_aligned_pic(erase_outside_arena(to_gray_enhanced(get_transformed_picture(pic_name,radius_ref,radius)),xyr),xyr)[0]

    # transforming the new arena pic to match the reference arena pic
    print('   * rescaling and filtering')
    img                  = get_transformed_picture(arena_pic_name) #edged.transform_pic_to_match_distance(arena_pic_ref,arena_pic_name,dx_ref,dy_ref,dx,dy)
    img_high_contrast_bw = to_gray_enhanced(img)
    img_mark             = get_transformed_picture(arena_pic_mark_name) if has_picture_with_marks else None

    ################
    ################
    ################
    # finding the center of the arena and shifting the new image to align the center with the ref image
    ##
    ##
    print('   * finding center and shifting')
    arena_xyr            = edged.find_arena_in_image(img_high_contrast_bw,img_mark,arena_color=mark_scheme.arena_boundary,dr=5,n_circle_points=100,circle_match_threshold=0.4)

    # the arena picture being cropped is zoomed in/out
    arena_radius_ref = 1.0
    arena_radius     = 1.0
    if abs(1.0 - arena_xyr[2]/arena_ref_xyr[2]) > arena_radius_tolerance:
        print(' ::: WARNING -> The arena is zoomed in/out! Trying to compensate that...')
        arena_radius_ref     = arena_ref_xyr[2]
        arena_radius         = arena_xyr[2]
        img                  = get_transformed_picture(arena_pic_name,arena_radius_ref,arena_radius) #edged.transform_pic_to_match_distance(arena_pic_ref,arena_pic_name,dx_ref,dy_ref,dx,dy)
        img_high_contrast_bw = to_gray_enhanced(img)
        img_mark             = get_transformed_picture(arena_pic_mark_name,arena_radius_ref,arena_radius) if has_picture_with_marks else None

    # trying again with the proper transform
    arena_xyr            = edged.find_arena_in_image(img_high_contrast_bw,mark_img=img_mark,arena_color=mark_scheme.arena_boundary,dr=5,n_circle_points=100,circle_match_threshold=0.4)
    img_shift,img_extent = get_cropped_aligned_pic(img,arena_xyr)

    # applying transforms to the supporting images
    img_mark             = get_cropped_aligned_pic(img_mark,arena_xyr)[0] if has_picture_with_marks else None
    img_entrance         = img_mark.copy() if has_picture_with_marks else (apply_all_transforms(arena_pic_entrance_name,arena_xyr,arena_radius_ref,arena_radius) if has_picture_with_entrance else None)
    img_find_holes       = img_mark.copy() if has_picture_with_marks else (apply_all_transforms(arena_pic_preproc_name, arena_xyr,arena_radius_ref,arena_radius) if has_picture_preproc       else img_high_contrast_bw.copy())
    
    ################ saving
    print('     ... saving: %s'%cropped_pic_name)
    img_shift.save(cropped_pic_name,dpi=img.info['dpi'])
    cropped_img_extent_str = '       - cropped image extent (cm) = ' + str(list(img_extent))

    ################
    ################
    ################
    # finding the holes of the arena
    ##
    ##
    print('   * finding hole and entrance positions')

    # setting up the transformation of coordinates to translate the holes and center into the world coordinates
    T_cropped          = misc.LinearTransf2D( (0,img_shift.size[0]), img_extent[:2], (img_shift.size[1],0), img_extent[2:])
    get_feat_xy_coord  = lambda xy_px: T_cropped(numpy.array(xy_px[:2]))
    get_all_xy_coords  = lambda coord_px,k0: numpy.array([ (k+k0,*get_feat_xy_coord(h)) for k,h in enumerate(coord_px) ])

    # finding entrances
    entrance_px     = []                      # just dummy values to hold the place and prevent crash
    entrance_coords = numpy.array([-1,-1,-1]) # just dummy values to hold the place and prevent crash
    if has_picture_with_marks or has_picture_with_entrance:
        entrance_color  = mark_scheme.entrances if has_picture_with_marks else None
        entrance_px     = edged.find_clusters(img_entrance,threshold=0.5,find_less_than_threshold=True,color=entrance_color)
        entrance_coords = get_all_xy_coords(entrance_px,-len(entrance_px))

    # finding targets
    targets_px     = []                      # just dummy values to hold the place and prevent crash
    targets_coords = numpy.array([-1,-1,-1]) # just dummy values to hold the place and prevent crash
    if has_picture_with_marks:
        if mark_scheme.IsField('targets'):
            target_color   = mark_scheme.targets if has_picture_with_marks else None
            targets_px     = edged.find_clusters(img_mark,threshold=0.5,find_less_than_threshold=True,color=target_color)
            targets_coords = get_all_xy_coords(targets_px,-len(entrance_px)-len(targets_px))
        else:
            print(' ***** to find the targets, you need a _mark.png picture from the original arena, marking the target with the mark_scheme.targets color')

    # finding holes
    holes_color  = mark_scheme.holes if has_picture_with_marks else None
    holes_px     = edged.find_clusters(img_find_holes,threshold=0.5,find_less_than_threshold=True,color=holes_color)
    holes_coords = get_all_xy_coords(holes_px,1)

    # drawing holes in the picture
    img_find_holes = img_find_holes.convert('RGB')
    draw_result = PIL.ImageDraw.Draw(img_find_holes)
    holes_color  = mark_scheme.holes if has_picture_with_marks else None
    for k,h in enumerate(holes_px):
        hole_radius = numpy.max((h[2],2))
        draw_result.ellipse((h[0]-hole_radius,h[1]-hole_radius,h[0]+hole_radius,h[1]+hole_radius),outline=(255,0,0),fill=None)
        draw_result.text(tuple(numpy.array(h[:2])+hole_radius/2),str(k+1),anchor='rb',fill=holes_color)
    entrance_color  = mark_scheme.entrances if has_picture_with_marks else (0,255,0)
    for k,h in enumerate(entrance_px):
        hole_radius = numpy.max((h[2],2))
        draw_result.ellipse((h[0]-hole_radius,h[1]-hole_radius,h[0]+hole_radius,h[1]+hole_radius),outline=entrance_color,fill=None)
        draw_result.text(tuple(numpy.array(h[:2])+hole_radius/2),str(k-len(entrance_px)),anchor='rb',fill=entrance_color)
    target_color   = mark_scheme.targets if has_picture_with_marks else (255,0,0)
    for k,h in enumerate(targets_px):
        hole_radius = numpy.max((h[2],2))
        draw_result.ellipse((h[0]-hole_radius,h[1]-hole_radius,h[0]+hole_radius,h[1]+hole_radius),outline=target_color,fill=None)
        draw_result.text(tuple(numpy.array(h[:2])+hole_radius/2),str(k-len(entrance_px)-len(targets_px)),anchor='rt',fill=target_color)
    
    # applying transformations to arena coords
    arena_center_world = list(misc.LinearTransf2D( (0,img.size[0]), dx, (img.size[1],0), dy)(numpy.array(arena_xyr[:2])).flatten())
    arena_R_world      = (misc.LinearTransf( (0.0,img.size[0]), (0.0,sum(numpy.abs(dx))) )(arena_xyr[2]) + misc.LinearTransf( (0.0,img.size[1]), (0.0,sum(numpy.abs(dy))) )(arena_xyr[2]))/2.0

    # writing hole txt file and picture
    print('     :::: NEW ARENA PICTURE INFO ::::')
    arena_info_txt  = '%s\n       - arena center (cm) = %s\n       - arena radius (cm) = %.8f'%(cropped_img_extent_str,str(arena_center_world),arena_R_world)
    arena_info_txt += '\n      indices ::: (a) below {0} == targets;    (b) [{0},...,-1] == entrances;    (c) 1... == holes'.format(-len(entrance_px))
    print(arena_info_txt)
    header_str = 'Hole coordinates for file %s\nCropped file = %s\nHole visualizaton = %s\n%s\nhole index,x (cm),y (cm)'%(arena_pic_name,cropped_pic_name,holes_pic_filename,arena_info_txt)
    print('     ... saving: %s'%holes_txt_filename)
    numpy.savetxt(holes_txt_filename,numpy.row_stack((targets_coords,entrance_coords,holes_coords)),fmt=['%d','%.8f','%.8f'],delimiter=',',header=header_str)
    img_find_holes.save(holes_pic_filename,dpi=img.info['dpi'])
    print('     ... saving: %s'%holes_pic_filename)

    # transforming the high contrast version of the picture (used to detect the arena) with the (0,0)_world in blue
    print('   * cropping the BW version of the arena')
    img_high_contrast_bw = img_high_contrast_bw.convert('RGB')
    draw_result = PIL.ImageDraw.Draw(img_high_contrast_bw)
    draw_result.ellipse((arena_xyr[0]-arena_xyr[2], arena_xyr[1]-arena_xyr[2], arena_xyr[0]+arena_xyr[2], arena_xyr[1]+arena_xyr[2]), outline=(255,0,0,0))
    draw_result.ellipse((arena_xyr[0]-2,arena_xyr[1]-2,arena_xyr[0]+2,arena_xyr[1]+2),fill=(0,0,255))
    img_high_contrast_bw_shift = get_cropped_aligned_pic(img_high_contrast_bw,arena_xyr)[0]
    print('     ... saving: %s'%origin_pic_name)
    img_high_contrast_bw_shift.save(origin_pic_name,dpi=img.info['dpi'])
    


 *** processing 2022/BKGDimage-220812.png
   * rescaling and filtering
   * finding center and shifting
     ... saving: arena_picture\2022/BKGDimage-220812_cropped.png
   * finding hole and entrance positions
     :::: NEW ARENA PICTURE INFO ::::
       - cropped image extent (cm) = [-102.8197797797798, 91.28792792792794, -70.57941071428571, 75.46916071428572]
       - arena center (cm) = [1.5131131131131212, 0.6192678571428729]
       - arena radius (cm) = 60.45233447
      indices ::: (a) below -4 == targets;    (b) [-4,...,-1] == entrances;    (c) 1... == holes
     ... saving: arena_picture\2022/BKGDimage-220812_holes.txt
     ... saving: arena_picture\2022/BKGDimage-220812_holes.png
   * cropping the BW version of the arena
     ... saving: arena_picture\2022/BKGDimage-220812_origin.png
 *** processing 2022/BKGDimage-20221011.png
   * rescaling and filtering
   * finding center and shifting
     ... saving: arena_picture\2022/BKGDimage-20221011_cropped.png
   * finding hole and e